In [11]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SVM libraries
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn.datasets import load_digits
from sklearn.metrics import classification_report

# Read data set
data = pd.read_excel('Raisin_Dataset.xlsx', sheet_name = 'Raisin_Dataset')
data

# Data frame structure
data.info()

# Convert columns to numeric
data['Class'] = data['Class'].str.replace('Kecimen', '0')
data['Class'] = data['Class'].str.replace('Besni', '1')

cols = data.columns
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')

# Data frame structure
data.info()

# Classes are distrubuted evenly
data['Class'].value_counts()

# Data for SVM
X = data
X = X.drop(['Class'], axis = 1)
Y = data.Class # class

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             900 non-null    int64  
 1   MajorAxisLength  900 non-null    float64
 2   MinorAxisLength  900 non-null    float64
 3   Eccentricity     900 non-null    float64
 4   ConvexArea       900 non-null    int64  
 5   Extent           900 non-null    float64
 6   Perimeter        900 non-null    float64
 7   Class            900 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 56.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             900 non-null    int64  
 1   MajorAxisLength  900 non-null    float64
 2   MinorAxisLength  900 non-null    float64
 3   Eccentricity     900 non-null    f

In [23]:
C = [1, 10, 100, 1000, 10000, 100000]
C = np.array(C)

Accuracy_CV_new = np.zeros((len(C),2))
Accuracy_rbf = np.zeros((k,2))

for k in range(1,10)
X_train_rbf, X_test_rbf, Y_train_rbf, Y_test_rbf = train_test_split(X, Y, test_size = 0.10) # 10 times cs so test_size is 10% of data set
    
# Standarization
sc = StandardScaler()
sc.fit(X_train_rbf)
X_train_rbf = sc.transform(X_train_rbf)
X_test_rbf = sc.transform(X_test_rbf)
X_train_rbf = pd.DataFrame(X_train_rbf)
X_test_rbf = pd.DataFrame(X_test_rbf)

for i in range(0, len(C)):
    # Model
    svclassifier = SVC(kernel = 'rbf', C = C[i], gamma = 0.05) # gamma constant
    svclassifier.fit(X_train_rbf, Y_train_rbf)
    y_pred_rbf = svclassifier.predict(X_test_rbf)
        
    # Accuracy
    Accuracy_CV_new[i,0] = sum(y_pred_rbf == Y_test_rbf)/len(Y_test_rbf)
    Accuracy_CV_new[i,1] = C[i]

Accuracy_rbf[k,0] = np.max(Accuracy_CV_new[:,0]) # best accuracy
id_max = np.argmax(Accuracy_CV_new[:,0])
Accuracy_rbf[k,1] = Accuracy_CV_new[id_max,1] # C for best acuracy